Organize Imports

In [1]:
import os
import sys
import importlib
import numpy as np
import tensorflow as tf
if '__file__' in locals():
    dir_path = os.path.dirname(os.path.realpath(__file__))
else:
    dir_path = os.getcwd()
python_path = dir_path + '/../python/'
sys.path.append(python_path)


import dataset_loader
importlib.reload(dataset_loader)
from dataset_loader import EM_DATA
from dataset_loader import BATCH_SIZE, NBOX_IN,NBOX_OUT,N_CHANNELS
import net_3d_1

Parameters 

In [2]:
N_EPOCHS = 30

INPUTS: FOLDERS 

In [3]:
#define folders
data_fld = "/Users/markroza/Documents/work_from_home/NNcourse_project/data/res6/synth/"
out_fld = "/Users/markroza/Documents/work_from_home/NNcourse_project/data/results/synth_res6/"

Create Database

In [4]:
data_pairs = dataset_loader.read_list_file(data_fld+'list.txt')
all_pdbs = [x[0] for x in  data_pairs]
em1 = EM_DATA(data_fld,train_pdbs=all_pdbs[1:10],test_pdbs=['5ydz'])
train_data = em1.train_dataset.make_initializable_iterator()
test_data = em1.test_dataset.make_initializable_iterator()
trn = train_data.get_next()
tst = test_data.get_next()




/Users/markroza/Documents/work_from_home/NNcourse_project/data/res6/synth//F_5vhh_output.npy


KeyError: 'OUT'

Create Network

In [ ]:
x = tf.placeholder(tf.float32, shape=[BATCH_SIZE, NBOX_IN, NBOX_IN, NBOX_IN,N_CHANNELS ])
y = tf.placeholder(tf.float32, shape=[BATCH_SIZE, NBOX_OUT, NBOX_OUT, NBOX_OUT,1 ])
keep_prob = tf.placeholder(dtype=tf.float32)
isTrain = tf.placeholder(dtype=tf.bool)

nn = net_3d_1.get_net_graph(x,y,keep_prob,isTrain)

In [ ]:
# open session and initialize all variables
config = tf.ConfigProto()
config.gpu_options.allow_growth = True

with tf.Session(config=config) as sess:
    #sess = tf.InteractiveSession()
    tf.global_variables_initializer().run()

    logger = tf.summary.FileWriter(out_fld+'/graphs', sess.graph)
    merged = tf.summary.merge_all()


    model_path = out_fld+'/network_test/'
    if os.path.isdir(model_path) is False:
        os.mkdir(model_path)
    saver = tf.train.Saver()
    
    for epoch_num in range(N_EPOCHS):
        sess.run(train_data.initializer)
        # training-loop
        for batch in range(em1.N_batches):
            for _ in range(4):
                image_in,image_out = sess.run(trn)
                sess.run(nn["D_optim"], feed_dict={x: image_in, y: image_out, keep_prob: 0.8, isTrain: True})
                sess.run(nn["clip"])
                loss_d_, loss_g_, _VAE_loss, _KL_divergence, _reconstruction_loss, summary,\
                _, _, _ = sess.run([nn["D_loss"], nn["G_loss"], nn["VAE_L"], nn["mean_KL"],\
                nn["mean_recon"], merged, nn["D_optim"], nn["G_optim"], nn["E_optim"]],\
                            feed_dict={x: image_in, y: image_out, keep_prob: 0.8, isTrain: True})
                sess.run(nn["clip"])
                if batch % 10 == 0:
                    print("batch:", batch)
                    print("D Loss:", loss_d_)
                    print("G Loss:", loss_g_)
                    print("VAE loss:", _VAE_loss)
                    print("KL divergence:", _KL_divergence)
                    print("reconstruction_loss:", _reconstruction_loss)
                    print("###########")
                    if batch % 500 == 0:
                        G = sess.run(nn["G_z"]["o"], feed_dict={x: image_in, y: image_out, keep_prob: 0.8, isTrain: False})
                        np.save(out_fld + str(batch) + ".npy", G)
                        if batch % 10 == 0:
                            saver.save(sess, model_path + str(batch) + ".ckpt")



In [ ]:
a = em1.train_points
print(a[0].ijk)
print(a[1].ijk)
print(a[2].ijk)

In [ ]:
b = np.random.permutation(a)
print(b[0].ijk)
print(b[1].ijk)
print(b[2].ijk)